<a href="https://colab.research.google.com/github/AI-Maker-Space/LLM-Ops-Vault/blob/main/Week%201/First%20Session/Barbie_Retrieval_Augmented_Question_Answering_(RAQA)_Assignment%20(Assignment%20Version).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Questioning Barbie Reviews with RAQA (Retrieval Augmented Question Answering)

In the following notebook, you are tasked with creating a system that answers questions based on information found in reviews of the 2023 Barbie movie.

## Build 🏗️

There are 3 main tasks in this notebook:

1. Obtain and parse reviews from a review website
2. Create a Vectorstore from the reviews
3. Create a `RetrievalQA` using the VectorStore

## Ship 🚢

Create a Hugging Face Space that hosts your application.

## Share 🚀

Make a social media post about your final application.

>### Why RAQA and not RAG?
>If we look at the original [paper](https://arxiv.org/abs/2005.11401), we find that RAG is a fairly specific and well defined term that isn't exactly the same as "retrieve context, feed context to model in the prompt".
>For that reason, we're making the decision to delineate between "actual" RAG, and Retrieval Augmented Question Answering - which is not a well defined phrase.

### Pre-task Work

All we really need to do to get started is to get our prerequisites!

We'll be leveraging `langchain` and `openai` today.

Check out the docs:
- [LangChain](https://docs.langchain.com/docs/)
- [OpenAI](https://github.com/openai/openai-python)

In [62]:
# !pip install -q -U openai langchain

In [64]:
import os
import getpass

os.environ["OPENAI_API_KEY"] = getpass.getpass("Open AI API Key:")

Open AI API Key:········


### Task 1: Data Preparation

In this task we'll be collecting, and then parsing, our data.

#### Scraping IMDB Reviews of Barbie

We'll use some Selenium based trickery to get the reviews we need to make our application.

Check out the docs here:
- [Selenium](https://www.selenium.dev/documentation/)

In [3]:
#!pip install -q -U requests

In [4]:
#!pip install -q -U scrapy selenium

In [5]:
#!apt install chromium-chromedriver

The operation couldn’t be completed. Unable to locate a Java Runtime.
Please visit http://www.java.com for information on installing Java.



In [6]:
import numpy as np
import pandas as pd
from scrapy.selector import Selector
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")

In [7]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome(options=chrome_options)

In [8]:
url = "https://www.imdb.com/title/tt1517268/reviews/?ref_=tt_ov_rt"
driver.get(url)

In [9]:
sel = Selector(text = driver.page_source)
review_counts = sel.css('.lister .header span::text').extract_first().replace(',','').split(' ')[0]
more_review_pages = int(int(review_counts)/25)

In [10]:
for i in tqdm(range(more_review_pages)):
    try:
        css_selector = 'load-more-trigger'
        driver.find_element(By.ID, css_selector).click()
    except:
        pass

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████| 46/46 [00:05<00:00,  9.03it/s]


In [29]:
rating_list = []
review_date_list = []
review_title_list = []
author_list = []
review_list = []
review_url_list = []
error_url_list = []
error_msg_list = []
reviews = driver.find_elements(By.CSS_SELECTOR, 'div.review-container')

for d in tqdm(reviews):
    try:
        sel2 = Selector(text = d.get_attribute('innerHTML'))
        try:
            rating = sel2.css('.rating-other-user-rating span::text').extract_first()
        except:
            rating = np.NaN
        try:
            review = sel2.css('.text.show-more__control::text').extract_first()
        except:
            review = np.NaN
        try:
            review_date = sel2.css('.review-date::text').extract_first()
        except:
            review_date = np.NaN
        try:
            author = sel2.css('.display-name-link a::text').extract_first()
        except:
            author = np.NaN
        try:
            review_title = sel2.css('a.title::text').extract_first()
        except:
            review_title = np.NaN
        try:
            review_url = sel2.css('a.title::attr(href)').extract_first()
        except:
            review_url = np.NaN
        rating_list.append(rating)
        review_date_list.append(review_date)
        review_title_list.append(review_title)
        author_list.append(author)
        review_list.append(review)
        review_url_list.append(review_url)
    except Exception as e:
        error_url_list.append(url)
        error_msg_list.append(e)
review_df = pd.DataFrame({
    'Review_Date':review_date_list,
    'Author':author_list,
    'Rating':rating_list,
    'Review_Title':review_title_list,
    'Review':review_list,
    'Review_Url':review_url
    })

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:01<00:00, 76.93it/s]


In [30]:
review_df.head()

,Review_Date,Author,Rating,Review_Title,Review,Review_Url
0,21 July 2023,LoveofLegacy,6,"Beautiful film, but so preachy\n","Margot does the best with what she's given, bu...",/review/rw9200852/?ref_=tt_urv
1,22 July 2023,imseeg,7,3 reasons FOR seeing it and 1 reason AGAINST.\n,The first reason to go see it:,/review/rw9200852/?ref_=tt_urv
2,22 July 2023,Natcat87,6,Too heavy handed\n,"As a woman that grew up with Barbie, I was ver...",/review/rw9200852/?ref_=tt_urv
3,31 July 2023,ramair350,10,"As a guy I felt some discomfort, and that's o...",As much as it pains me to give a movie called ...,/review/rw9200852/?ref_=tt_urv
4,24 July 2023,heatherhilgers,9,A Technicolor Dream\n,"Wow, this movie was a love letter to cinema. F...",/review/rw9200852/?ref_=tt_urv


Let's save this `pd.DataFrame` as a `.csv` to our local session (this will be terminated when you terminate the Colab session) so we can leverage it in LangChain!

Check out the docs if you get stuck:
- [`to_csv()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.to_csv.html)

In [31]:
### YOUR CODE HERE

review_df.to_csv('barbie.csv', index=False)



#### Data Parsing

Now that we have our data - let's go ahead and start parsing it into a more usable format for LangChain!

We'll be using the `CSVLoader` for this application.

We also want to be sure to track the sources that our review came from!

Check out the docs here:
- [`CSVLoader`](https://python.langchain.com/docs/integrations/document_loaders/csv)

In [32]:
from langchain.document_loaders.csv_loader import CSVLoader

In [33]:
loader = CSVLoader(
    file_path="barbie.csv",
    source_column="Review_Url"
    )

data = loader.load()

In [34]:
print(data)

[Document(page_content='Review_Date: 21 July 2023\nAuthor: LoveofLegacy\nRating: 6\nReview_Title: Beautiful film, but so preachy\nReview: Margot does the best with what she\'s given, but this film was very disappointing to me. It was marketed as a fun, quirky satire with homages to other movies. It started that way, but ended with over-dramatized speeches and an ending that clearly tried to make the audience feel something, but left everyone just feeling confused. And before you say I\'m a crotchety old man, I\'m a woman in my 20s, so I\'m pretty sure I\'m this movie\'s target audience. The saddest part is there were parents with their kids in the theater that were victims of the poor marketing, because this is not a kid\'s movie. Overall, the humor was fun on occasion and the film is beautiful to look at, but the whole concept falls apart in the second half of the film and becomes a pity party for the "strong" woman.\nReview_Url: /review/rw9200852/?ref_=tt_urv', metadata={'source': '/

In [37]:
len(data)

100

In [38]:
assert len(data) == 100

Now that we have collected our review information into a loader - we can go ahead and chunk the reviews into more manageable pieces.

We'll be leveraging the `RecursiveCharacterTextSplitter` for this task today.

While splitting our text seems like a simple enough task - getting this correct/incorrect can have massive downstream impacts on your application's performance.

You can read the docs here:
- [RecursiveCharacterTextSplitter](https://python.langchain.com/docs/modules/data_connection/document_transformers/text_splitters/recursive_text_splitter)

We want to split our documents into 1000 character length chunks, with 100 characters of overlap.

> ### HINT:
>It's always worth it to check out the LangChain source code if you're ever in a bind - for instance, if you want to know how to transform a set of documents, check it out [here](https://github.com/langchain-ai/langchain/blob/5e9687a196410e9f41ebcd11eb3f2ca13925545b/libs/langchain/langchain/text_splitter.py#L268C18-L268C18)

In [41]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 100, # the character length of the chunk
    chunk_overlap = 100 , # the character length of the overlap between chunks
    length_function = len , # the length function
)

In [43]:
documents = text_splitter.split_documents(data)

In [44]:
print(documents)

[Document(page_content='Review_Date: 21 July 2023\nAuthor: LoveofLegacy\nRating: 6', metadata={'source': '/review/rw9200852/?ref_=tt_urv', 'row': 0}), Document(page_content='Author: LoveofLegacy\nRating: 6\nReview_Title: Beautiful film, but so preachy', metadata={'source': '/review/rw9200852/?ref_=tt_urv', 'row': 0}), Document(page_content="Review: Margot does the best with what she's given, but this film was very disappointing to me. It", metadata={'source': '/review/rw9200852/?ref_=tt_urv', 'row': 0}), Document(page_content="Margot does the best with what she's given, but this film was very disappointing to me. It was", metadata={'source': '/review/rw9200852/?ref_=tt_urv', 'row': 0}), Document(page_content="does the best with what she's given, but this film was very disappointing to me. It was marketed as", metadata={'source': '/review/rw9200852/?ref_=tt_urv', 'row': 0}), Document(page_content="the best with what she's given, but this film was very disappointing to me. It was markete

In [45]:
len(documents)

5650

In [47]:
assert len(documents) == 5650

With our documents transformed into more manageable sizes, and with the correct metadata set-up, we're now ready to move on to creating our VectorStore!

### Task 2: Creating an "Index"

The term "index" is used largely to mean: Structured documents parsed into a useful format for querying, retrieving, and use in the LLM application stack.

#### Selecting Our VectorStore

There are a number of different VectorStores, and a number of different strengths and weaknesses to each.

In this notebook, we will be keeping it very simple by leveraging [Facebook AI Similarity Search](https://ai.meta.com/tools/faiss/#:~:text=FAISS%20(Facebook%20AI%20Similarity%20Search,more%20scalable%20similarity%20search%20functions.), or `FAISS`.

In [48]:
!pip install -q -U faiss-cpu tiktoken

We're going to be setting up our VectorStore with the OpenAI embeddings model. While this embeddings model does not need to be consistent with the LLM selection, it does need to be consistent between embedding our index and embedding our queries over that index.

While we don't have to worry too much about that in this example - it's something to keep in mind for more complex applications.

We're going to leverage a [`CacheBackedEmbeddings`](https://python.langchain.com/docs/modules/data_connection/caching_embeddings )flow to prevent us from re-embedding similar queries over and over again.

Not only will this save time, it will also save us precious embedding tokens, which will reduce the overall cost for our application.

>#### Note:
>The overall cost savings needs to be compared against the additional cost of storing the cached embeddings for a true cost/benefit analysis. If your users are submitting the same queries often, though, this pattern can be a massive reduction in cost.

In [53]:
# from langchain.embeddings.openai import OpenAIEmbeddings
# from langchain.embeddings import CacheBackedEmbeddings
# from langchain.vectorstores import FAISS
# from langchain.storage import LocalFileStore

# store = ### YOUR CODE HERE

# core_embeddings_model = ### YOUR CODE HERE

# embedder = CacheBackedEmbeddings.from_bytes_store(
#     ### YOUR CODE HERE,
#     ### YOUR CODE HERE,
#     ### YOUR CODE HERE
# )

# vector_store = ### YOUR CODE HERE




from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.embeddings import CacheBackedEmbeddings
from langchain.vectorstores import FAISS
from langchain.storage import LocalFileStore

store = LocalFileStore("./cache/")

core_embeddings_model = OpenAIEmbeddings()

embedder = CacheBackedEmbeddings.from_bytes_store(
    core_embeddings_model,
    store,
    namespace=core_embeddings_model.model
)

vector_store = FAISS.from_documents(documents, embedder)

Now that we've created the VectorStore, we can check that it's working by embedding a query and retrieving passages from our reviews that are close to it.

In [54]:
query = "How is Will Ferrell in this movie?"
embedding_vector = core_embeddings_model.embed_query(query)
docs = vector_store.similarity_search_by_vector(embedding_vector, k = 4)

for page in docs:
  print(page.page_content)

was the Will Ferrell character and his board that made their point but then became superfluos.
the movie and they are, but Will Ferrell, Simu Liu, and the whole rest of the cast were also great
9.5 because of two flaws: First was the Will Ferrell character and his board that made their point
film was very disjointed. Ryan Gosling as good as he is seemed to old to play Ken and Will Ferrell


Let's see how much time the `CacheBackedEmbeddings` pattern saves us:

In [55]:
%%timeit -n 1 -r 1
query = "I really wanted to enjoy this and I know that I am not the target audience but there were massive plot holes and no real flow."
embedding_vector = core_embeddings_model.embed_query(query)
docs = vector_store.similarity_search_by_vector(embedding_vector, k = 4)

183 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [56]:
%%timeit
query = "I really wanted to enjoy this and I know that I am not the target audience but there were massive plot holes and no real flow."
embedding_vector = core_embeddings_model.embed_query(query)
docs = vector_store.similarity_search_by_vector(embedding_vector, k = 4)

166 ms ± 53.1 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


As we can see, even over a significant number of runs - the cached query is significantly faster than the first instance of the query!

With that, we're ready to move onto Task 3!

### Task 3: Building a Retrieval Chain

In this task, we'll be making a Retrieval Chain which will allow us to ask semantic questions over our data.

This part is rather abstracted away from us in LangChain and so it seems very powerful.

Be sure to check the documentation, the source code, and other provided resources to build a deeper understanding of what's happening "under the hood"!

#### A Basic RetrievalQA Chain

We're going to leverage `return_source_documents=True` to ensure we have proper sources for our reviews - should the end user want to verify the reviews themselves.

Hallucinations [are](https://arxiv.org/abs/2202.03629) [a](https://arxiv.org/abs/2305.15852) [massive](https://arxiv.org/abs/2303.16104) [problem](https://arxiv.org/abs/2305.18248) in LLM applications.

Though it has been tenuously shown that using Retrieval Augmentation [reduces hallucination in conversations](https://arxiv.org/pdf/2104.07567.pdf), one sure fire way to ensure your model is not hallucinating in a non-transparent way is to provide sources with your responses. This way the end-user can verify the output.

#### Our LLM

In this notebook, we'll continue to leverage OpenAI's suite of models - this time we'll be using the `gpt-3.5-turbo` model to power our RetrievalQAWithSources chain.

Check out the relevant documentation if you get stuck:
- [`OpenAIChat()`](https://python.langchain.com/docs/modules/model_io/models/chat/)

In [57]:
from langchain.llms.openai import OpenAIChat

llm = OpenAIChat(model="gpt-3.5-turbo")

Now we can set up our chain.

We'll need to make our VectorStore a retriever in order to be able to leverage it in our chain - let's check out the `as_retriever()` method.

Relevant Documentation:
- [`FAISS`](https://api.python.langchain.com/en/latest/vectorstores/langchain.vectorstores.faiss.FAISS.html#langchain.vectorstores.faiss.FAISS.as_retriever)

In [58]:
retriever = vector_store.as_retriever()

In [59]:
from langchain.chains import RetrievalQA
from langchain.callbacks import StdOutCallbackHandler

handler = StdOutCallbackHandler()

qa_with_sources_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    callbacks=[handler],
    return_source_documents=True
)

In [60]:
qa_with_sources_chain({"query" : "How was Will Ferrell in this movie?"})



> Entering new RetrievalQA chain...

> Finished chain.


{'query': 'How was Will Ferrell in this movie?',
 'result': 'The answer is not clear based on the given context.',
 'source_documents': [Document(page_content='was the Will Ferrell character and his board that made their point but then became superfluos.', metadata={'source': '/review/rw9200852/?ref_=tt_urv', 'row': 76}),
  Document(page_content='9.5 because of two flaws: First was the Will Ferrell character and his board that made their point', metadata={'source': '/review/rw9200852/?ref_=tt_urv', 'row': 76}),
  Document(page_content='film was very disjointed. Ryan Gosling as good as he is seemed to old to play Ken and Will Ferrell', metadata={'source': '/review/rw9200852/?ref_=tt_urv', 'row': 67}),
  Document(page_content='the movie and they are, but Will Ferrell, Simu Liu, and the whole rest of the cast were also great', metadata={'source': '/review/rw9200852/?ref_=tt_urv', 'row': 10})]}

In [61]:
qa_with_sources_chain({"query" : "Do reviewers consider this movie Kenough?"})



> Entering new RetrievalQA chain...

> Finished chain.


{'query': 'Do reviewers consider this movie Kenough?',
 'result': 'Based on the given information, it can be inferred that the movie is considered "Kenough" by at least three out of the four reviewers mentioned, as their review titles include the words "Kenough" or "Ken."',
 'source_documents': [Document(page_content='Review_Date: 23 July 2023\nAuthor: eoinageary\nRating: 8\nReview_Title: I am Kenough', metadata={'source': '/review/rw9200852/?ref_=tt_urv', 'row': 39}),
  Document(page_content='Review_Date: 20 July 2023\nAuthor: Genti25\nRating: 8\nReview_Title: You are Kenough', metadata={'source': '/review/rw9200852/?ref_=tt_urv', 'row': 24}),
  Document(page_content='Review_Date: 22 July 2023\nAuthor: finnconnelly-63017\nRating: 10\nReview_Title: Ken out of ten', metadata={'source': '/review/rw9200852/?ref_=tt_urv', 'row': 18}),
  Document(page_content='Author: brianjohnson-20043\nRating: 4\nReview_Title: This movie tries to be too much', metadata={'source': '/review/rw9200852/?ref_=

And with that, we have our Barbie Review RAQA Application built!

### Conclusion

Now that we have our application ready, let's deploy it to a Hugging Face space with their Docker spaces.

You can find the next part [here](https://ai-maker-space-barbie-raqa-application-chainlit-demo.hf.space/readme)!

You've built, now it's time to ship! 🚢